# Run the speciation model with fastscape

In [1]:
import numpy as np
import pandas as pd
import xsimlab as xs
import matplotlib.pyplot as plt

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

from fastscape.models import basic_model
from fastscape.processes import MultipleFlowRouter

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from paraspec.fastscape_ext import IR12Speciation, IR12EnvironmentElevation

## Speciation without competition in an evolving landscape

In [2]:
model = basic_model.update_processes({
    'life_env': IR12EnvironmentElevation,
    'life': IR12Speciation,
    'flow': MultipleFlowRouter
})

model

<xsimlab.Model (18 processes, 19 inputs)>
grid
    shape              [in] ('shape_yx',) nb. of grid nodes in (y, x)
    length             [in] ('shape_yx',) total grid length in (y, x)
boundary
    status             [in] () or ('border',) node status at borders
fs_context
uplift
    rate               [in] () or ('y', 'x') uplift rate
tectonics
init_topography
    seed               [in] random seed
surf2erode
diffusion
    diffusivity        [in] () or ('y', 'x') diffusivity (transport...
init_erosion
flow
    slope_exp          [in] () or ('y', 'x') MFD partioner slope ex...
drainage
spl
    k_coef             [in] () or ('y', 'x') bedrock channel incisi...
    area_exp           [in] drainage area exponent
    slope_exp          [in] slope exponent
erosion
vmotion
topography
terrain
life_env
life
    init_size          [in] initial population size
    init_min_trait     [in] initial min trait value
    init_max_trait     [in] initial max trait value
    random_seed        [in] ra

In [3]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.linspace(0., 1e6, 1001),
        'out': np.linspace(0., 1e6, 101)
    },
    master_clock='time',
    input_vars={
        'grid__shape': [201, 201],
        'grid__length': [2e4, 2e4],
        'boundary__status': ['looped', 'looped', 'fixed_value', 'fixed_value'],
        'uplift__rate': 1e-3,
        'flow__slope_exp': 1.,
        'spl__k_coef': 1e-4,
        'diffusion__diffusivity': 1e-3,
        'life': {
            'init_size': 100,
            'init_min_trait': 0,
            'init_max_trait': 100,
            'nb_radius': 1e3,
            'capacity': 15,
            'sigma_d': 100.,
            'sigma_mut': 25,
            'sigma_w': 50,
            'random_seed': 1234
        }
    },
    output_vars={
        'topography__elevation': 'out',
        'drainage__area': 'out',
        'life__x': 'out',
        'life__y': 'out',
        'life__opt_trait': 'out',
        'life__trait': 'out',
        'life__parent': 'out',
        'life__id': 'out'
    }
)

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=model)

             0% | initialize 

In [ ]:
df = (
    out_ds
    # only keep paraspec output variables
    .drop_dims([d for d in out_ds.dims if d not in ('out', 'pop')])
    .drop_vars(in_ds, errors='ignore')
    # convert to a dataframe
    # see https://xarray-simlab.readthedocs.io/en/latest/io_storage.html#dynamically-sized-arrays
    .stack(fpop=('out', 'pop'))
    .dropna('fpop', how='all')
    .to_dataframe()
    # clean
    .rename(columns=lambda name: name.replace('life__', ''))
    .reset_index()
    .dropna()
    .set_index('id', drop=False)
    # assign tree-specific extra columns
    .assign(
        tree_id=lambda df: df.ptree.tree_id,
        node_type=lambda df: df.ptree.node_type
    )
)


df

## Plot population

In [ ]:
times = np.arange(0, 1e6, 2.5e5)
elev_max = out_ds.topography__elevation.max()


fg = (out_ds
      .sel(out=times)
      .topography__elevation.plot(col='out', col_wrap=2, figsize=(13, 10)))


for ax, t in zip(fg.axes.ravel(), times):
    pop = df[df.out==t]
    ax.scatter(pop.x, pop.y, c=pop.trait, vmin=0, vmax=elev_max, s=20, edgecolor='w')

## Plot subtree 

In [ ]:
def plot_tree(G, pos=None, root=None, alpha=0.5, node_size=40,
              node_color='trait', cmap=plt.cm.viridis,
              edge_color=None, edge_cmap=plt.cm.gray,
              vmin=None, vmax=None, prog='dot', ax='none', fig='none'):

    if pos is None:
        pos = graphviz_layout(G, prog=prog, root=root)
    else:
        x, y = pos
        pos = {k: (v[x], v[y]) for k, v in G.nodes.data()}

    node_data = [n.get(node_color) for n in G.nodes.values()]
    if vmin is None:
        vmin = np.nanmin(node_data)
    if vmax is None:
        vmax = np.nanmax(node_data)

    if edge_color is None:
        edge_data = 'k'
    else:
        edge_data = [e.get(edge_color) for e in G.edges.values()]

    if ax=='none':
        fig, ax = plt.subplots(figsize=(14, 12))


    nx.draw(G, pos=pos, ax=ax, alpha=alpha,
            node_size=node_size, node_color=node_data,
            cmap=cmap, vmin=vmin, vmax=vmax,
            edge_color=edge_data, edge_cmap=edge_cmap)

    sm = plt.cm.ScalarMappable(cmap=cmap,
                               norm=plt.Normalize(vmin=vmin, vmax=vmax))

    sm.set_array([])
    
    cbar = plt.colorbar(sm, fraction=0.025, pad=0.04)

    return fig, ax

In [ ]:
tree_id = df.tree_id.value_counts().idxmax()  # find root with largest tree
leaf = df[(df.tree_id==tree_id) & (df.out==df.out.max())].id  # end nodes (leaf) of tree_id

# define subtree (dataframe and networkx)
df_tree = (
    df
    .ptree.merge_forest(-1)
    .ptree.drop_disconnected_roots()
    .ptree.extract_subtree(leaf=list(leaf))
)

G = df_tree.ptree.to_networkx(node_col='trait')

In [ ]:
plot_tree(G, prog='dot', node_color='trait', node_size=10, cmap=plt.cm.viridis,
          vmin=0, vmax=elev_max);

## Speciation with competition in an evolving landscape

In [ ]:
from paraspec.fastscape_ext import DD03Speciation, DD03EnvironmentElevation

In [ ]:
model2 = basic_model.update_processes({
    'life_env': DD03EnvironmentElevation,
    'life': DD03Speciation,
    'flow': MultipleFlowRouter
})
model2

In [ ]:
in_ds2 = xs.create_setup(
    model=model2,
    clocks={
        'time': np.linspace(0., 1e6, 1001),
        'out': np.linspace(0., 1e6, 101)
    },
    master_clock='time',
    input_vars={
        'grid__shape': [201, 201],
        'grid__length': [2e4, 2e4],
        'boundary__status': ['looped', 'looped', 'fixed_value', 'fixed_value'],
        'uplift__rate': 1e-3,
        'flow__slope_exp': 1.,
        'spl__k_coef': 1e-4,
        'diffusion__diffusivity': 1e-3,
        'life': {
            'init_size': 100,
            'init_min_trait': 0.5,
            'init_max_trait': 0.5,
            'birth_rate': 1.,
            'movement_rate': 5.,
            'slope_topt_env': 0.95,
            'car_cap_max': 500.,
            'sigma_opt_trait': 0.3,
            'mut_prob': 0.005,
            'sigma_mut': 0.05,
            'sigma_mov': 0.12e3,
            'sigma_comp_trait': 0.9,
            'sigma_comp_dist': 0.19,
            'random_seed': 1234
        }
    },
    output_vars={
        'topography__elevation': 'out',
        'drainage__area': 'out',
        'life__x': 'out',
        'life__y': 'out',
        'life__trait': 'out',
        'life__parent': 'out',
        'life__id': 'out'
    }
)

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds2 = in_ds2.xsimlab.run(model=model2)

In [ ]:
df2 = (
    out_ds2
    # only keep paraspec output variables
    .drop_dims([d for d in out_ds2.dims if d not in ('out', 'pop')])
    .drop_vars(in_ds2, errors='ignore')
    # convert to a dataframe
    # see https://xarray-simlab.readthedocs.io/en/latest/io_storage.html#dynamically-sized-arrays
    .stack(fpop=('out', 'pop'))
    .dropna('fpop', how='all')
    .to_dataframe()
    # clean
    .rename(columns=lambda name: name.replace('life__', ''))
    .reset_index()
    .dropna()
    .set_index('id', drop=False)
    # assign tree-specific extra columns
    #.assign(
    #    tree_id=lambda df: df.ptree.tree_id,
    #    node_type=lambda df: df.ptree.node_type
    #)
)


df2

In [ ]:
times = np.arange(0, 1e6, 2.5e5)
elev_max = out_ds2.topography__elevation.max()


fg = (out_ds2
      .sel(out=times)
      .topography__elevation.plot(col='out', col_wrap=2, figsize=(13, 10)))


for ax, t in zip(fg.axes.ravel(), times):
    pop = df2[df2.out==t]
    ax.scatter(pop.x, pop.y, c=pop.trait, vmin=0, vmax=1, s=20, edgecolor='w')